<a href="https://colab.research.google.com/github/omniaghazy/Core_X/blob/main/Dampjng_loss.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [21]:
!pip install pybullet

In [22]:
import pybullet as p
import time
import pybullet_data
import math
import pandas as pd
import os
import numpy as np # ضروري لو استخدمنا math.sin
from typing import List, Dict # عشان شكل الـCode يكون منظم

print("Libraries installed and imported successfully.")

Libraries installed and imported successfully.


In [23]:
# ==========================================================
# ⚙️ (ملغي) الـBacklash Injector مش محتاجينه ⚙️
# ==========================================================

# ==========================================================
# الخطوة 2: إعداد بيئة المحاكاة (زي ما هو)
# ==========================================================
# (هنا هييجي الـSetup بتاع الـconnect والـloadURDF)
print("\nStarting PyBullet simulation in HEADLESS (DIRECT) mode...")
# ... (نفس الـCode بتاع الـSetup لغاية ما الـrobotId يتحمل)

physicsClient = p.connect(p.DIRECT)
pybullet_data_path = pybullet_data.getDataPath()
p.setAdditionalSearchPath(pybullet_data_path)
p.setGravity(0, 0, -9.81)
p.setRealTimeSimulation(0)
planeId = p.loadURDF("plane.urdf")
startPos = [0, 0, 0.01]
startOrientation = p.getQuaternionFromEuler([0, 0, 0])
# *ملاحظة: لو الـKUKA ما تحملش، هنغيره للـR2D2 زي ما اتفقنا قبل كده*
robotId = p.loadURDF("kuka_iiwa/model.urdf", startPos, startOrientation, useFixedBase=True)
print(f"Successfully loaded KUKA LBR iiwa (Robot ID: {robotId})")


Starting PyBullet simulation in HEADLESS (DIRECT) mode...
Successfully loaded KUKA LBR iiwa (Robot ID: 5)


In [24]:
# --- 4. إعداد المفاصل ---
numJoints = p.getNumJoints(robotId)
controllable_joints = [i for i in range(numJoints) if p.getJointInfo(robotId, i)[2] == p.JOINT_REVOLUTE]
end_effector_link_index = 6
for joint in controllable_joints:
    # (جديد) تفعيل الـForce/Torque Sensor عشان نسجل الـTorque اللي هيزيد
    p.enableJointForceTorqueSensor(robotId, joint, enableSensor=1)
    p.setJointMotorControl2(robotId, joint, p.VELOCITY_CONTROL, force=0)

In [27]:
import pybullet as p
import time
import pybullet_data
import math
import pandas as pd
import os
import numpy as np # للتأكد من وجودها لو math.sin مش شغالة (بس هنا الـmath شغالة)
from typing import List, Dict # عشان شكل الـCode يكون منظم

# ==========================================================
# ⚙️ (ملغي) الـBacklash Injector مش محتاجينه في المشكلة دي ⚙️
# (هنا مكان الكود بتاع الـSetup والـConnect اللي بيطلع robotId)
# ==========================================================

# (نحتاج نتأكد إن الـCode ده موجود قبل الـLoop)
# تعطيل التحكم الافتراضي وتفعيل الـSensors
# for joint in controllable_joints:
#     p.enableJointForceTorqueSensor(robotId, joint, enableSensor=1)
#     p.setJointMotorControl2(robotId, joint, p.VELOCITY_CONTROL, force=0)

# --------------------------------------------------------------------------

# --- 5. إعداد "حقن العطل" (Damping Loss) ---
joint_to_fail = 4
INITIAL_DAMPING = 0.5
MIN_DAMPING = 0.01
DAMPING_DECREMENT = (INITIAL_DAMPING - MIN_DAMPING) / 10000.0

data_log: List[List] = []
total_steps = 20000
fault_injection_step = 10000

print(f"Simulation starting for {total_steps} steps (Damping Loss)...")
print(f" - FAULT (Damping Decrease) will be injected in JOINT {joint_to_fail} starting at step {fault_injection_step}.")

# --- 6. اللوب الرئيسي للمحاكاة (توليد البيانات) ---
start_time = time.time()
try:
    current_damping = INITIAL_DAMPING # نبدأ بالتخميد الطبيعي

    # (جديد) دي لـstorage مؤقت عشان الـLoop الداخلية
    perfect_commands_list = []

    for step in range(total_steps):

        # أ. حساب "المسار" المستهدف (Trajectory)
        target_x = 0.6 + 0.2 * math.sin(step * 0.01)
        target_y = 0.0
        target_z = 0.8
        target_position = [target_x, target_y, target_z]

        # ب. حساب الأوامر المثالية (Inverse Kinematics)
        perfect_commands_list = p.calculateInverseKinematics(
            robotId, end_effector_link_index, target_position
        )

        # --- ج. (حقن العطل) ---
        if step >= fault_injection_step:
            if step == fault_injection_step:
                print(f"\n[STEP {step}] --- FAULT INJECTED! Starting Damping Decrease for JOINT {joint_to_fail} ---\n")

            # 1. تقليل قيمة التخميد تدريجياً
            current_damping -= DAMPING_DECREMENT

            # 2. تطبيق الـChange Dynamics (باستخدام jointDamping)
            p.changeDynamics(
                bodyUniqueId=robotId,
                linkIndex=joint_to_fail,
                jointDamping=max(current_damping, MIN_DAMPING)
            )

        log_row = [step] # بداية صف البيانات

        # --- د. لوب على كل المفاصل للتحكم والتسجيل ---
        for i in range(len(controllable_joints)):
            joint_index = controllable_joints[i]

            # 1. استخراج الأمر المثالي
            target_pos = perfect_commands_list[i]

            # 2. إرسال الأمر للمحاكي
            p.setJointMotorControl2(
                bodyIndex=robotId,
                jointIndex=joint_index,
                controlMode=p.POSITION_CONTROL,
                targetPosition=target_pos,
                positionGain=0.1,
                velocityGain=0.5
            )

            # 3. تسجيل البيانات
            joint_state = p.getJointState(robotId, joint_index)
            actual_position = joint_state[0]
            actual_velocity = joint_state[1] # الـSignal الأهم هنا

            # إضافة الـData للـRow
            log_row.append(target_pos)
            log_row.append(actual_position)
            log_row.append(actual_velocity)


        # تسجيل الـLabel النهائي للخطوة
        log_row.append(current_damping) # بنسجل قيمة الـDamping الحالية كـLabel
        data_log.append(log_row) # إضافة الصف بالكامل للـData Log
        p.stepSimulation() # تشغيل خطوة المحاكاة

except Exception as e:
    print(f"\nAn error occurred during simulation: {e}")
finally:
    # --- 7. إنهاء الاتصال وحفظ البيانات ---
    end_time = time.time()
    p.disconnect()
    print("Simulation disconnected.")
    print(f"Total simulation time: {end_time - start_time:.2f} seconds.")

    if data_log:
        print(f"Logged {len(data_log)} data points.")
        columns = ['Step']
        for j in controllable_joints:
            columns.append(f'Cmd_J{j}')
            columns.append(f'Act_J{j}')
            columns.append(f'Vel_J{j}') # عمود جديد للـVelocity
        columns.append('Damping_Label')

        df = pd.DataFrame(data_log, columns=columns)
        output_filename = "trajectory_Damping_loss_data.csv"
        df.to_csv(output_filename, index=False)

        print("\n========================================================")
        print(f"✅ SUCCESS! Data saved to: {os.path.abspath(output_filename)}")
        print(f"Columns: {columns}")
    else:
        print("No data was logged.")

Simulation starting for 20000 steps (Damping Loss)...
 - FAULT (Damping Decrease) will be injected in JOINT 4 starting at step 10000.

[STEP 10000] --- FAULT INJECTED! Starting Damping Decrease for JOINT 4 ---

Simulation disconnected.
Total simulation time: 62.98 seconds.
Logged 20000 data points.

✅ SUCCESS! Data saved to: /content/trajectory_Damping_loss_data.csv
Columns: ['Step', 'Cmd_J0', 'Act_J0', 'Vel_J0', 'Cmd_J1', 'Act_J1', 'Vel_J1', 'Cmd_J2', 'Act_J2', 'Vel_J2', 'Cmd_J3', 'Act_J3', 'Vel_J3', 'Cmd_J4', 'Act_J4', 'Vel_J4', 'Cmd_J5', 'Act_J5', 'Vel_J5', 'Cmd_J6', 'Act_J6', 'Vel_J6', 'Damping_Label']


In [29]:
df = pd.read_csv("/content/trajectory_Damping_loss_data.csv")
df.head()

,Step,Cmd_J0,Act_J0,Vel_J0,Cmd_J1,Act_J1,Vel_J1,Cmd_J2,Act_J2,Vel_J2,...,Cmd_J4,Act_J4,Vel_J4,Cmd_J5,Act_J5,Vel_J5,Cmd_J6,Act_J6,Vel_J6,Damping_Label
0,0,-3.914271e-12,0.000000,0.000000,0.501474,0.000000,0.0,-1.241552e-12,0.000000,0.0,...,8.906893e-13,0.000000,0.000000,0.169378,0.000000,0.000000,0.000000,0.000000,0.000000,0.5
1,1,-3.500491e-01,-0.416667,-100.000000,0.397104,-0.416667,-100.0,5.248216e-01,0.416667,100.0,...,7.421969e-02,0.069687,16.724954,0.274285,0.057057,13.693637,0.004735,0.004735,1.136514,0.5
2,2,3.278852e-03,0.000000,100.000000,0.305680,-0.833333,-100.0,5.506677e-03,0.000000,-100.0,...,-9.561443e-02,-0.101619,-41.113438,0.599241,0.473723,100.000000,0.367056,0.367056,86.956837,0.5
3,3,-7.517362e-02,0.416667,100.000000,0.540281,-0.416667,100.0,1.199341e-01,0.416667,100.0,...,2.040942e-01,0.315048,100.000000,0.755859,0.471589,-0.512226,-0.046922,-0.046922,-99.354678,0.5
4,4,6.534144e-01,0.675055,62.013082,1.056129,0.000000,100.0,1.286059e+00,0.833333,100.0,...,7.116855e-01,0.731715,100.000000,0.537730,0.593373,29.228115,-0.162244,-0.162244,-27.677317,0.5
